In [ ]:
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import os

In [ ]:
def estimate_pitch (estimates):
    estimates = sorted(estimates)
    X = np.array(estimates).reshape(-1, 1)
    clustering = AgglomerativeClustering(n_clusters=None, 
                                         affinity='euclidean', 
                                         linkage='average', 
                                         distance_threshold=5).fit(X)
    clusters = []
    for i in range(np.max(clustering.labels_)+1):
        ccount = 0
        csum = 0
        for j, e in enumerate(clustering.labels_):
            if e == i:
                ccount += 1
                csum += estimates[j] 
        clusters.append((ccount, csum / ccount))
    clusters.sort(reverse=True)
    return clusters[0][1]

In [ ]:
def find_optimal_pitch (current_collections, collection, part):
    pitch_estimates = []
    time = current_collections[collection]['boersma'][part][0]
    
    for algo in algorithms:
        if part in current_collections[collection][algo]:
            pitch_estimates.append(current_collections[collection][algo][part][1])    
            
            if len(current_collections[collection][algo][part][0]) < len(time):
                time = current_collections[collection][algo][part][0]
    
    best_estimate = np.empty(len(time))
    for i in range(len(time)):
        current_pitches = []
        for pitches in pitch_estimates:
            current_pitches.append(pitches[i])
        best_estimate[i] = estimate_pitch(current_pitches)
        
    return (time, best_estimate)

### Scherbaum Mshavanadze

In [ ]:
collections = {
    "GVM009": {},
    "GVM017": {},
    "GVM019": {},
    "GVM031": {},
    "GVM097": {}
}

algorithms = ['boersma', 'crepe', 'hermes', 'maddox', 'noll', 'praat']
data_dir = '/Akamai/voice/data/pitches-vuv/'

for collection in collections:
    for algo in algorithms:
        collections[collection][algo] = {}
        
def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

for algorithm in os.listdir(data_dir):
    if not algorithm in algorithms:
        continue
    for collection in os.listdir(f"{data_dir}{algorithm}"):
        if collection != 'Scherbaum Mshavanadze':
            continue
        for song in os.listdir(f"{data_dir}{algorithm}/{collection}"):
            for part in os.listdir(f"{data_dir}{algorithm}/{collection}/{song}"):
                print(part)
                if part[:6] in collections:
                    if 'AHDS' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('AHDS'):part.index('AHDS')+6]] = (x, y)
                    elif 'ALRX' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('ALRX'):part.index('ALRX')+6]] = (x, y)
                    elif 'AOLS' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('AOLS'):part.index('AOLS')+6]] = (x, y)
                    elif 'VSOA' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('VSOA'):part.index('VSOA')+6]] = (x, y)

In [ ]:
parts = ['AHDS1M', 'AHDS2M', 'AHDS3M', 'ALRX1M', 'ALRX2M', 'ALRX3M', 'VSOAX4', 'AOLS5S']

res_dir = '/Akamai/voice/data/ground-estimate/Scherbaum Mshavanadze/'
for collection in collections:
    for part in parts:
        t, estimate = find_optimal_pitch(collections, collection, part)
        if not os.path.isdir(res_dir + collection):
            os.mkdir(res_dir + collection)
        np.savetxt(res_dir + collection + '/' + part + '.txt', np.c_[t, estimate], delimiter=' ', fmt='%f')
        print('done')

### Teach Yourself Megrelian Songs

In [ ]:
collections2 = {
    "Ak'a Si Rekisho": {},
    'Gepshvat Ghvini': {},
    'Io _ Chkin Kiana': {},
    'Mesishi Vardi': {},
    'Meureme': {},
    'Mi Re Sotsodali_': {},
    "Mole Chit'i Gilakhe": {},
    'O Da': {},
    'Vojanudi Chkim Jargvals': {}
}

algorithms = ['boersma', 'crepe', 'hermes', 'maddox', 'noll']
data_dir = '/Akamai/voice/data/pitches-vuv/'

for collection in collections2:
    for algo in algorithms:
        collections2[collection][algo] = {}
        
def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

def root_name (name):
    s = None; e = None
    for i in range(len(name)-2, 0, -1):
        if name[i] == '.': e = i;
        elif name[i:i+2] == '_A': 
            s = i
            break
    
    if s == None: return name[:e];
    else: return name[:s];

for algorithm in os.listdir(data_dir):
    if not algorithm in algorithms:
        continue
    for collection in os.listdir(f"{data_dir}{algorithm}"):
        if collection != 'Teach Yourself Megrelian Songs':
            continue
        for song in os.listdir(f"{data_dir}{algorithm}/{collection}"):
            for part in os.listdir(f"{data_dir}{algorithm}/{collection}/{song}"):
                if root_name(part) in collections2:
                    if 'AHDS' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}", algorithm)
                        collections2[root_name(part)][algorithm][part[part.index('AHDS'):part.index('AHDS')+6]] = (x, y)
                    else:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}", algorithm)
                        collections2[root_name(part)][algorithm]['VSOAX4'] = (x, y)

In [ ]:
parts = ['AHDS1M', 'AHDS2M', 'AHDS3M', 'VSOAX4']

res_dir = '/Akamai/voice/data/ground-estimate/Teach Yourself Megrelian Songs/'
for collection in collections2:
    for part in parts:
        t, estimate = find_optimal_pitch(collections2, collection, part)
        if not os.path.isdir(res_dir + collection):
            os.mkdir(res_dir + collection)
            
        np.savetxt(res_dir + collection + '/' + part + '.txt', np.c_[t, estimate], delimiter=' ', fmt='%f')

### Teach Yourself Gurian Songs

In [ ]:
collections3 = {
    "Adila-Alipasha": {},
    "Indi-Mindi": {},
    'Mival Guriashi (1)': {} ,
    'Pikris Simghera': {},
    "Alaverdi": {},
    "K'alos Khelkhvavi": {},
    'Mival Guriashi (2)' : {}, 
    "Sabodisho": {},
    "Khasanbegura": {},     
    "Mok'le Mravalzhamieri": {},
    'Sadats Vshobilvar': {},
    "Beri Ak'vans Epareba": {}, 
    "Lat'aris Simghera": {},    
    "Mts'vanesa Da Ukudosa": {}, 
    "Shermanduli": {},
    "Brevalo": {},             
    "Manana": {},         
    'Nanina (1)': {},      
    "Shvidk'atsa": {},
    "Chven-Mshvidoba": {},    
    "Maq'ruli": {},               
    'Nanina (2)': {},          
    'Supris Khelkhvavi': {},
    'Didi Khnidan': {},     
    "Masp'indzelsa Mkhiarulsa": {}, 
    "Orira": {},                
    "Ts'amok'ruli": {},
    "Gakhsovs, T'urpa": {},
    "Me-Rustveli": {},        
    "P'at'ara Saq'varelo": {}
}

algorithms = ['boersma', 'crepe', 'hermes', 'maddox', 'noll']
data_dir = '/Akamai/voice/data/pitches-vuv/'

for collection in collections3:
    for algo in algorithms:
        collections3[collection][algo] = {}
        
def separate(adir, algo):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

def root_name (name):
    s = None; e = None
    for i in range(len(name)-2, 0, -1):
        if name[i] == '.': e = i;
        elif name[i:i+2] == '_A': 
            s = i
            break
    
    if s == None: return name[:e];
    else: return name[:s];

for algorithm in os.listdir(data_dir):
    if not algorithm in algorithms:
        continue
    for collection in os.listdir(f"{data_dir}{algorithm}"):
        if collection != 'Teach Yourself Gurian Songs':
            continue
        for song in os.listdir(f"{data_dir}{algorithm}/{collection}"):
            for part in os.listdir(f"{data_dir}{algorithm}/{collection}/{song}"):
                print(root_name(part))
                if root_name(part) in collections3:
                    if 'AHDS' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}", algorithm)
                        collections3[root_name(part)][algorithm][part[part.index('AHDS'):part.index('AHDS')+6]] = (x, y)
                    else:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}", algorithm)
                        collections3[root_name(part)][algorithm]['VSOAX4'] = (x, y)

In [ ]:
parts = ['AHDS1M', 'AHDS2M', 'AHDS3M', 'VSOAX4']

res_dir = '/Akamai/voice/data/ground-estimate/Teach Yourself Gurian Songs/'
for collection in collections3:
    for part in parts:
        t, estimate = find_optimal_pitch(collections3, collection, part)
        if not os.path.isdir(res_dir + collection):
            os.mkdir(res_dir + collection)
            
        np.savetxt(res_dir + collection + '/' + part + '.txt', np.c_[t, estimate], delimiter=' ', fmt='%f')
        print('done')

### Testing and Debugging

In [ ]:
data_dir = '/Akamai/voice/data/ground-estimate/Scherbaum Mshavanadze/'
data_dir += os.listdir(data_dir)[0] + '/'
data_dir += os.listdir(data_dir)[0]

x, y = separate(data_dir)
plt.figure(figsize=(20, 5))
plt.plot(x, y, '.', markersize=2)

In [ ]:
estimate_pitch([201.9599, 201.3181, 201.1465, 200.005, 67.73271])

In [ ]:
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt

In [ ]:
pitches = [201.9599, 201.3181, 201.1465, 200.005, 67.73271, 65.234, 90.23084]
X = np.array(pitches).reshape(-1, 1)
Z = hierarchy.linkage(X, 'average')
plt.figure()
dn = hierarchy.dendrogram(Z)
plt.ylim(0, 10)

print(Z)

In [ ]:
def recurse_find (start_index, N, Z):
    ans = []
    
    x = int(Z[start_index][0])
    y = int(Z[start_index][1])
    w = Z[start_index][2]
    
    if x < N: ans.append(x)
    if y < N: ans.append(y)
        
    if x >= N: ans += recurse_find(x - N, N, Z)
    if y >= N:ans += recurse_find(y - N, N, Z)
    
    return ans

def traverse_dendogram (N, Z):   
    clusters = set([0, 1, 2, 3, 4])
    
    for i in range(Z.shape[0]):
        x = int(Z[i][0])
        y = int(Z[i][1])
        w = Z[i][2]
        
    return cluster_values
    

# a = traverse_dendogram(hierarchy.linkage(X, 'average'))
recurse_find(2, len(pitches), Z)

In [ ]:
def calculate_labels (estimates):
    current_threshold = 5
    X = np.array(estimates).reshape(-1, 1)
    current_clustering = None
    
    while True:
        aglo_clust = AgglomerativeClustering(n_clusters=None, 
                                         affinity='euclidean', 
                                         linkage='average', 
                                         distance_threshold=current_threshold)
        current_clustering = aglo_clust.fit(X)
        stats = Counter(current_clustering.labels_).most_common()
        
        if len(stats) >= 2 and stats[0][1] == stats[1][1]:
            current_threshold += 1
        else: break
            
    return current_clustering.labels_
            

def estimate_pitch (estimates):
    estimates = sorted(estimates)
    clustering = calculate_labels(estimates)
    print(estimates)
    print(clustering)
    clusters = []
    for i in range(np.max(clustering)+1):
        ccount = 0
        csum = 0
        for j, e in enumerate(clustering):
            if e == i:
                ccount += 1
                csum += estimates[j] 
        clusters.append((ccount, csum / ccount))
    clusters.sort(reverse=True)
    return clusters[0][1]

In [ ]:
pitches = [201.9599, 201.3181, 201.1465, 200.005, 67.73271, 65.234, 68.322, 69.212]
X = np.array(pitches).reshape(-1, 1)
Z = hierarchy.linkage(X, 'average')
print(estimate_pitch(pitches))
print(sorted(pitches))

In [ ]:
from collections import Counter

In [ ]:
Counter([0, 0, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1,]).most_common()